## Краткое описание решения

### Общий план
Я выделил несколько ключевых идей, на которые опирался при решениии:  
1. **Сделать локальную валидацию, которая будет коррелировать с лидербордом.**  
Это даст неограниченное количество проверок своих решений. И какую-то информацию о данных с теста: если обычные методы валидации дают качественный результат, то в тесте нет каких-то подвохов от организаторов.  
Финальное разбиение: кросс-валидация для отбора фич (70%) + валидация для проверки решений (20%) + отложенная выборка для финального скора (10%)
2. **Провести Adversarial Validation.**  
Это также может показать наличие подводных камней в данных. Дополнительно я проверю отсутствие дата ликов в фичах, которые я мог бы упустить.
3. **Написать качественные фичи.**   
На это опирается мое решение. Я не нашел каких-то ликов в анализе данных, поэтому решил подумать, что было бы наиболее информативным для модели. 
4. **Работать с градиентным бустингом.**
Эту модель я регулярно использую в работе, поэтому хорошо с ней знаком. Дополнительный плюс - он умеет работать с пропущенными значениями.


### Признаки
Мои топ фичи основаны на знании рейтинга игрока в следующей игре. Я пришел к этому следующим путём:  
При анализе данных на трейне и на тесте я заметил, что нет какого-то определенного фактора, который мог бы разделить эти выборки. То есть разбиение данных происходило случайно. При этом мы знаем рейтинги игроков. А значит, можно попробовать написать алгоритм, который по фиче X21 (время игры - game_time) находил бы рейтинг каждого игрока на момент следующей игры. Тогда мы могли бы сделать однозначный вывод о том, кто победил, а кто проиграл.  
Но реальность оказалась более сложной. Сложности с которыми я столкнулся:  
1. Некоторые игроки проводили несколько игр в один game_time. При этом в каких-то он побеждал, а в каких-то проигрывал. *Решение: усреднять рейтинг игрока по значению game_time.*    
2. Для некоторых игроков во всем датасете всего одна запись. *Решение: считать, что признак принимает значение nan в такие моменты.*  
3. Основная проблема - победа не всегда означала повышение рейтинга. *Решение: отдать это на откуп бустингу :) Подумал, что стоит сначала посмотреть на результаты прогнозов. Они оказались достаточно хороши, и на этом я остановился.*  
4. Я брал информацию по всему датасету. Так делать не совсем правильно в общем случае. Потому что мы используем ту информацию, которой в реальной жизни знать бы не могли. *Решение: в описании и чате было разрешено использовать всё, поэтому я проигнорировал свои сомнения. Тем более, что проверка на дата лики показала, что всё ок*

### Остальные фичи
Предварительно я написал несколько других фич, которые не имеют большого смысла, но могли бы либо выявить лики в данных (напр. разница в id игроков), либо как-то неявно агрегировать информацию, которую я в ходе анализа данных не нашел (напр. средний номер юнита). В итоге график важности фичей по метрике gain показывает, что основной вклад вносят фичи, связанные с рейтингом в следующей игре. Поэтому нет смысла останавливаться на этом подробно.  

Идеи, которые я не реализовал:
- отбор фич. Уверен, что на одних фичах по рейтингу можно получить результат не хуже. Но решил, что можно не тратить время на это.  
- фичи, связанные с процентом побед игрока. Тут у меня получился даталик, так как на тесте скорее всего есть игроки, которых не было на трейне. И там фича давала бы лик. Не стал этим заниматься, потому что были другие идеи, которые я хотел проверить.  
- target encoding. Никогда не применял эту технику, но всегда хотел. Уверен, что при правильной реализации это могло бы дать очень хороший результат в данной задаче. Но я посчитал важным сфокусироваться на других аспектах.  
- составление характеристик каждого юнита. Ещё одна идея, которая кажется перспективной, но требует слишком много времени на анализ. Думаю, что если бы контест шел ещё неделю, то я бы её реализовал. К тому же на это дан намёк в описании задания.   

### Финал
Таким был мой общий ход решения. Спасибо большое за организацию соревнования. Было приятно применить свои навыки в новой для себя задаче. Наверняка я упустил какие-то очевидные моменты в данных, поэтому буду рад узнать об этом в каких-то дополнительных материалах о задаче.  
Мои результаты:  
- 0.565142 -- кросс-валидация  
- 0.56496 -- валидация  
- 0.56541 -- отложенная выборка  
- 0.56521 и пятое место -- приватный лидерборд

In [ ]:
# imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import log_loss
from sklearn.linear_model import LogisticRegression
from tqdm.auto import tqdm
import lightgbm as lgb
from copy import deepcopy as dp
from collections import defaultdict

# pandas setting
pd.options.display.max_columns = 50
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [ ]:
# support functions

In [ ]:
def split_data(X: pd.DataFrame, y: pd.DataFrame, cv_frac: float = 0.7, valid_frac: float = 0.2, test_frac: float = 0.1) -> list:
    '''
    Splits data into three parts + stratification.
    Output:
    tuple : Three datasets and targets
    '''
    assert round(cv_frac + valid_frac + test_frac, 5) == 1, f'{cv_frac + valid_frac + test_frac}'
    X_cv, X_oof, y_cv, y_oof = train_test_split(X, y, train_size=cv_frac, 
                                                shuffle=True, random_state=1, stratify=y)
    X_valid, X_test, y_valid, y_test = train_test_split(X_oof, y_oof, train_size=(valid_frac / (valid_frac + test_frac)), 
                                                        shuffle=True, random_state=1, stratify=y_oof)
    return X_cv, y_cv, X_valid, y_valid, X_test, y_test

In [ ]:
def predict(fitted_model, X_test: pd.DataFrame, y_test: pd.Series) -> list:
    '''
    Generate predict and calc score.
    Output:
    tuple : predictions, score
    '''
    preds = fitted_model.predict_proba(X_test)
    score = log_loss(y_test, preds)
    score = round(score, 5)
    return preds, score

In [ ]:
def fit_model(X_train: pd.DataFrame, y_train: pd.Series, X_test: pd.DataFrame, y_test: pd.Series, model, model_params: dict = {}):
    '''
    Initialization + fitting + results on train and test.
    Output:
    dict : model, preds on train and test, target, scorec on train and test
    '''
    curr_model = model(**model_params)
    curr_model.fit(X_train, y_train)
    
    preds_train, train_score = predict(curr_model, X_train, y_train)
    preds_test, test_score = predict(curr_model, X_test, y_test)
    
    return {'model': curr_model, 'preds': preds_test, 'y_true': y_test, 'test_score': test_score, 'train_score': train_score}

In [ ]:
def get_cross_val_res(X: pd.DataFrame, y: pd.Series, model, model_params: dict, n_splits:int = 5) -> list:
    '''
    Perform cross-validation.
    Output:
    list : model, preds on train and test, target, scorec on train and test
    '''
    skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=0)
    res = []
    for fold_num, (train_index, test_index) in tqdm(enumerate(skf.split(X, y)), total=n_splits):
        print('Current fold:', fold_num)
        X_train, X_test = X.iloc[train_index], X.iloc[test_index]
        y_train, y_test = y.iloc[train_index], y.iloc[test_index]

        output = fit_model(X_train, y_train, X_test, y_test, model, params)
        res.append(output)
        print(output['train_score'], output['test_score'], end='\n\n')
    return res

In [ ]:
def get_cross_val_score(res: list, score_col: str = 'test_score') -> float:
    '''
    Calc score for CV.
    Output:
    float : mean score over all folds
    '''
    scores = np.mean([i[score_col] for i in res])
    return scores    

In [ ]:
data_folder = '../raw_data/'
train_file = 'train.csv'
test_file = 'test.csv'

In [ ]:
train_df = pd.read_csv(f'{data_folder}/{train_file}')
test_df = pd.read_csv(f'{data_folder}/{test_file}')

## Data preprocessing

In [ ]:
# rename columns for better understandig
new_col_names = ['id', 'fight_type', 'p1', 'p1_rating', 'p2', 'p2_rating', 
                    'p1_u1', 'p1_u2', 'p1_u3', 'p1_u4', 'p1_u5', 'p1_u6', 'p1_u7', 'p1_u8', 
                    'p2_u1', 'p2_u2', 'p2_u3', 'p2_u4', 'p2_u5', 'p2_u6', 'p2_u7', 'p2_u8', 
                    'game_time', 'target']

In [ ]:
train_df.columns = new_col_names
test_df.columns = new_col_names[:-1]

In [ ]:
train_df.head(2)

In [ ]:
# check nan values
train_df.isna().sum().sum()

In [ ]:
test_df.isna().sum().sum()

In [ ]:
# prepare data for future features

In [ ]:
players_info = []
for df in [train_df, test_df]:
    player_info_1 = df[['p1', 'p1_rating', 'game_time', 'fight_type']].copy()
    if 'target' in df:
        player_info_1['is_win'] = df['target'].apply(lambda x: x == 1)
    else:
        player_info_1['is_win'] = np.nan
    player_info_2 = df[['p2', 'p2_rating', 'game_time', 'fight_type']].copy()
    if 'target' in df:
        player_info_2['is_win'] = df['target'].apply(lambda x: x == 0)
    else:
        player_info_2['is_win'] = np.nan
    player_info_1.columns = ['p_id', 'rating', 'game_time', 'fight_type', 'is_win']
    player_info_2.columns = ['p_id', 'rating', 'game_time', 'fight_type', 'is_win']
    player_info_full = pd.concat([player_info_1, player_info_2], axis=0, ignore_index=True)
    players_info.append(dp(player_info_full))

In [ ]:
players_info_full = pd.concat(players_info, axis=0, ignore_index=True).sort_values(by='game_time')

In [ ]:
# sample
players_info_full[players_info_full['p_id'] == 1]

In [ ]:
# aggregate info about each player
player_info_dict = defaultdict(dict)
for player_id, rating, game_time, fight_type, is_win in tqdm(players_info_full.to_numpy()):
    player_id, rating, game_time, fight_type = int(player_id), int(rating), int(game_time), int(fight_type)
    if player_id not in player_info_dict:
        player_info_dict[player_id]['game_time'] = []
        player_info_dict[player_id]['rating'] = []
        player_info_dict[player_id]['fight_type'] = []
        player_info_dict[player_id]['is_win'] = []
    player_info_dict[player_id]['game_time'].append(game_time)
    player_info_dict[player_id]['rating'].append(rating)
    player_info_dict[player_id]['fight_type'].append(fight_type)
    player_info_dict[player_id]['is_win'].append(is_win)

In [ ]:
# sample
player_info_dict[1]

In [ ]:
%%time
# ETA ~5 mins, sorry
# aggregate info about each game and calc next game rating 
for key, info in tqdm(player_info_dict.items(), total=len(player_info_dict)):
    player_info_dict[key]['win_rate'] = np.nanmean(info['is_win'])
    player_info_dict[key]['avg_rating'] = np.nanmean(info['rating'])
    player_info_dict[key]['min_rating'] = np.min(info['rating'])
    player_info_dict[key]['max_rating'] = np.max(info['rating'])
    
    game_times = info['game_time']
    ratings = info['rating']
    
    avg_rating = defaultdict(list)  # {game_id: [all ratings for game_id]}
    for index in range(len(game_times)):
        game_id = game_times[index]
        rating = ratings[index]
        avg_rating[game_id].append(rating)
        
        
    next_game_ratings = {}
    for index in range(len(avg_rating)):
        if index + 1 == len(avg_rating):  
            curr_game = list((avg_rating.keys()))[index]
            next_game_rating = np.nan  # don't have better ideas
            next_game_ratings[curr_game] = next_game_rating
        else:
            curr_game = list((avg_rating.keys()))[index]
            next_game = list((avg_rating.keys()))[index + 1]
            next_game_rating = np.nanmean(avg_rating[next_game])
            next_game_ratings[curr_game] = next_game_rating
    
    
    player_info_dict[key]['next_game_ratings'] = next_game_ratings  # {game_id: next_game_id_rating}   
    

In [ ]:
# samle
player_info_dict[1]

## Data Split

In [ ]:
X = train_df.drop(columns=['target'])
y = train_df['target']

In [ ]:
X_cv, y_cv, X_valid, y_valid, X_test, y_test = split_data(X, y)

In [ ]:
X_cv.shape,  X_valid.shape,  X_test.shape, 

In [ ]:
y_cv.shape, y_valid.shape, y_test.shape

## Calc features

In [ ]:
class FeatureMaker():
    '''
    Calc all features.
    Input:
    ----------
    init_data : pd.DataFrame
        Raw data: train or test
    Output:
    pd.DataFrame : DataFrame with all features 
    '''
    def calc_base_features(self, init_data: pd.DataFrame) -> pd.DataFrame:
        '''Basic information'''
        new_features = pd.get_dummies(init_data['fight_type'], prefix='fight_type')
        new_features['game_time'] = init_data['game_time']
        
        return new_features
    
    def calc_unit_features(self, init_data: pd.DataFrame) -> pd.DataFrame:
        '''Unit-related features'''
        p1_units = ['p1_u1', 'p1_u2', 'p1_u3', 'p1_u4', 'p1_u5', 'p1_u6', 'p1_u7', 'p1_u8']
        p2_units = ['p2_u1', 'p2_u2', 'p2_u3', 'p2_u4', 'p2_u5', 'p2_u6', 'p2_u7', 'p2_u8']
        all_unit_columns = p1_units + p2_units
        
        new_features = pd.DataFrame(index=init_data.index)
        
        # trash-features
        new_features['p1_mean_unit'] = init_data[p1_units].mean(axis=1)
        new_features['p2_mean_unit'] = init_data[p2_units].mean(axis=1)
        new_features['p1_median_unit'] = init_data[p1_units].median(axis=1)
        new_features['p2_median_unit'] = init_data[p2_units].median(axis=1)
        new_features['p1_std_unit'] = init_data[p1_units].std(axis=1)
        new_features['p2_std_unit'] = init_data[p2_units].std(axis=1)
        new_features['p1_max_unit'] = init_data[p1_units].max(axis=1)
        new_features['p2_max_unit'] = init_data[p2_units].max(axis=1)
        new_features['p1_min_unit'] = init_data[p1_units].min(axis=1)
        new_features['p2_min_unit'] = init_data[p2_units].min(axis=1)
        
        new_features['mean_unit_diff'] = new_features['p1_mean_unit'] - new_features['p2_mean_unit']
        new_features['mean_unit_ratio'] = new_features['p1_mean_unit'] / new_features['p2_mean_unit']
        
        new_features['std_unit_diff'] = new_features['p1_std_unit'] - new_features['p2_std_unit']
        new_features['std_unit_ratio'] = new_features['p1_std_unit'] / new_features['p2_std_unit']
        
        new_features['max_unit_diff'] = new_features['p1_max_unit'] - new_features['p2_max_unit']
        new_features['max_unit_ratio'] = (new_features['p1_max_unit'] + 1) / (new_features['p2_max_unit'] + 1)
        
        new_features['min_unit_diff'] = new_features['p1_min_unit'] - new_features['p2_min_unit']
        new_features['min_unit_ratio'] = (new_features['p1_min_unit'] + 1) / (new_features['p2_min_unit'] + 1)
                    
        return new_features
    
    def calc_player_features(self, init_data: pd.DataFrame) -> pd.DataFrame:
        '''Player-related features'''
        new_features = pd.DataFrame(index=init_data.index)
        # trash features 
        new_features['id_diff'] = init_data['p1'] - init_data['p2']
        return new_features
    
    def calc_rating_features(self, init_data: pd.DataFrame) -> pd.DataFrame:
        '''Rating features. Here we are going to use player_info_dict data'''
        new_features = pd.DataFrame(index=init_data.index)
        new_features['rating_diff'] = init_data['p1_rating'] - init_data['p2_rating']
        new_features['rating_diff'] = new_features['rating_diff'].apply(lambda x: max(min(-42, x), 42))
        
        
        # player_info_dict features:
        for player in ['p1', 'p2']:
            new_features[f'{player}_avg_rating'] = init_data[player].apply(lambda x: player_info_dict[x]['avg_rating'])
            new_features[f'{player}_min_rating'] = init_data[player].apply(lambda x: player_info_dict[x]['min_rating'])
            new_features[f'{player}_max_rating'] = init_data[player].apply(lambda x: player_info_dict[x]['max_rating'])
            new_features[f'{player}_rating_after_game'] = init_data[[player, 'game_time']].apply(lambda x: player_info_dict[x[player]]['next_game_ratings'][x['game_time']], axis=1)
            new_features[f'{player}_rating_diff'] = new_features[f'{player}_rating_after_game'] - init_data[f'{player}_rating']
        new_features['avg_rating_diff'] = new_features['p1_avg_rating'] - new_features['p2_avg_rating']
        new_features['min_rating_diff'] = new_features['p1_min_rating'] - new_features['p2_min_rating']
        new_features['max_rating_diff'] = new_features['p1_max_rating'] - new_features['p2_max_rating']
        new_features['rating_after_game_diff'] = new_features['p1_rating_after_game'] - new_features['p2_rating_after_game']
        new_features['rating_diff_diff'] = new_features['p1_rating_diff'] - new_features['p2_rating_diff']
        
        return new_features
    
    def fit_transform(self, data: pd.DataFrame) -> pd.DataFrame or list:
        base_features = self.calc_base_features(data)
        unit_features = self.calc_unit_features(data)
        player_features = self.calc_player_features(data)
        rating_features = self.calc_rating_features(data)
        
        new_features = pd.concat([base_features, unit_features, player_features, rating_features], axis=1)
#         assert new_features.isna().sum().sum() == 0
        assert new_features.shape[0] == data.shape[0]
        
        return new_features
    


In [ ]:
fm = FeatureMaker()

In [ ]:
%%time
X_cv_features = fm.fit_transform(X_cv)

In [ ]:
X_valid_features = fm.fit_transform(X_valid)

In [ ]:
X_test_features = fm.fit_transform(X_test)

## Perform CV (You can skip this part)

In [ ]:
model = lgb.LGBMClassifier
params = {
    "boosting_type": "dart",
    "learning_rate": 0.1,
    "max_depth": 6,
    "num_leaves" : 40,
    "drop_rate": 0.7,
    "skip_drop": 0.7,
    "max_drop": 1,
    "verbosity": -1,
    "seed": 42,
    "n_jobs": 10,
    "n_estimators": 100
                }


In [ ]:
res = get_cross_val_res(X_cv_features, y_cv, model, params)

In [ ]:
get_cross_val_score(res)

In [ ]:
fitted_model = res[0]['model']

In [ ]:
lgb.plot_importance(fitted_model, importance_type='gain', figsize=(10, 20))

# Get valid score

In [ ]:
model

In [ ]:
params

In [ ]:
output = fit_model(X_cv_features, y_cv, X_valid_features, y_valid, model, params)

In [ ]:
output['test_score']

# Get test score

In [ ]:
_, score = predict(output['model'], X_test_features, y_test)

In [ ]:
score

# Make final predict

In [ ]:
X_prod_features = fm.fit_transform(test_df)

### Data leak test

In [ ]:
full_df = pd.concat([X_cv_features, X_prod_features], axis=0, ignore_index=True)

In [ ]:
target = pd.concat([pd.Series(1, index=X_cv.index), pd.Series(0, index=X_prod_features.index)], ignore_index=True)

In [ ]:
res = get_cross_val_res(full_df, target, model, params)

In [ ]:
from sklearn.metrics import roc_auc_score

In [ ]:
roc_auc_score(res[0]['y_true'], res[0]['preds'][:, 1])

In [ ]:
# no leaks if features so far!

### Predict and save

In [ ]:
preds = output['model'].predict_proba(X_prod_features)

In [ ]:
preds = preds[:, 1]

In [ ]:
submit_file = 'sample_submission.csv'

In [ ]:
submit = pd.read_csv(data_folder + submit_file)

In [ ]:
submit.shape[0] == preds.shape[0]

In [ ]:
submit['target'] = preds

In [ ]:
save_folder = '../submissions/'
save_name = 'more_rating_features.csv'

In [ ]:
submit.to_csv(save_folder + save_name, index=False, header=True)